In [ ]:
from types import SimpleNamespace
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from tqdm import tqdm

# Problem 1

In [ ]:
np.random.seed(0)

In [ ]:
# Constants
T = 50
C = 1
Q = 1
R = 1
theta = 1

In [ ]:
# State space model

def step_x(x, Q, **kwargs):
    return np.cos(kwargs["theta"] * x) + np.random.normal(0, np.sqrt(Q), size=x.shape)

def step_y(x, C, R, **kwargs):
    return C * x + np.random.normal(0, np.sqrt(R), size=x.shape)


# # State space model

# def step_x(x, Q, **kwargs):
#     return kwargs["theta"] * x + np.random.normal(0, np.sqrt(Q), size=x.shape)

# def step_y(x, C, R, **kwargs):
#     return C * x + np.random.normal(0, np.sqrt(R), size=x.shape)

In [ ]:
# Simulate data

def simulate_ssm(T, theta):
    x = np.zeros(T + 1)
    y = np.zeros(T + 1)
    x[0] = np.random.normal(0, 1)
    y[0] = x[0] + np.random.normal(0, 1)
    for t in range(1, T + 1):
        x[t] = step_x(x[t-1], Q, theta=theta)
        y[t] = step_y(x[t], C, R, theta=theta)
    return x[1:], y[1:]

In [ ]:
x_data, y_data = simulate_ssm(T, theta)
x_data.shape

In [ ]:
plt.plot(x_data)
plt.ylabel("$x_t$")
plt.xlabel("$t$");

In [ ]:
plt.plot(x_data)
plt.plot(y_data)

In [ ]:
# Fully Adapted Particle Filter (to estimate the likelihood z_hat = p(y_data|theta))

def fully_adapted_pf(initial_particles, step_x, C, Q, R, seed=0, **step_kwargs):
    """Fully adapted particle filter for a nonlinear Gaussian State Space Model.

    The importance weights are uniform for this PF.
    """
    if seed is not None:
        np.random.seed(seed)

    N = len(initial_particles)
    print(f"Running with {N} particles")
    particles = [None] * T + [initial_particles]  # draw initial particles - put at index -1
    nu_weights = [None] * T  # these are nu weights
    mean_observation = [None] * T  # p(y_t|x_t)
    std_observation = [None] * T
    mean_state_prediction = [None] * T  # p(x_t|x_t-1)
    std_state_prediction = [None] * T
    mean_filtering = [None] * T  # p(x_t|x_t-1, y_t)
    std_filtering = [None] * T
    ancestor_indices = [None] * T
    loglikelihood = 0

    K = Q * C / (C * Q * C + R)
    state_proposal_stddev = np.sqrt((1 - K * C) * Q)
    obs_proposal_stddev = np.sqrt(C * Q * C + R)

    for t in tqdm(range(T)):
        # WEIGHT
        # measurement
        fcn = step_x(particles[t-1], Q, **step_kwargs)
        obs_mean = C * fcn
        measurement_proposal_dist = scipy.stats.norm(obs_mean, obs_proposal_stddev)  # p(y_t|x_t-1)

        # compute weights (nu)
        log_nu_weights_unnorm = measurement_proposal_dist.logpdf(y_data[t])
        log_nu_weights_max = np.max(log_nu_weights_unnorm)
        nu_weights_unnorm = np.exp(log_nu_weights_unnorm - log_nu_weights_max) + log_nu_weights_max
        nu_weights[t] = nu_weights_unnorm / np.sum(nu_weights_unnorm)

        # RESAMPLE
        a_indices = np.random.choice(range(N), p=nu_weights[t], replace=True, size=N)
        ancestor_indices[t] = a_indices

        # PROPAGATE
        # state
        fcn = step_x(particles[t-1][a_indices], Q, **step_kwargs)
        state_mean = fcn + K * (y_data[t] - C * fcn)
        state_proposal_dist = scipy.stats.norm(state_mean, state_proposal_stddev)  # p(x_t|x_t-1^a_t,y_t)
        particles[t] = state_proposal_dist.rvs()

        # Store some statistics
        # marginal filtering mean and variance
        mean_filtering[t], std_filtering[t] = np.mean(particles[t]), np.std(particles[t])
        # prediction
        fcn = step_x(particles[t-1], Q, **step_kwargs)  # this is done before resampling
        state_prediction_dist = scipy.stats.norm(fcn, np.sqrt(Q))  # p(x_t|x_t-1)
        mean_state_prediction[t] = np.mean(state_prediction_dist.mean())
        std_state_prediction[t] = np.mean(state_prediction_dist.std())
        # measurement
        measurement_dist = scipy.stats.norm(C * particles[t], np.sqrt(R))
        mean_observation[t] = np.mean(measurement_dist.mean())
        std_observation[t] = np.mean(measurement_dist.std())

        # likelihood
        log_obs = measurement_dist.logpdf(y_data[t])
        log_state_prop = state_proposal_dist.logpdf(particles[t])
        log_state_pred = state_prediction_dist.logpdf(particles[t])
        loglikelihood += log_obs + log_state_pred - log_state_prop - log_nu_weights_unnorm - np.log(N)

    nu_weights = np.array(nu_weights)
    #particles = np.array(particles[-1:] + particles[1:-1])  # move initial particle to index 0  #  np.array(particles[:-1])
    particles = np.array(particles[:-1])  # remove initial state
    mean_filtering = np.array(mean_filtering)
    std_filtering = np.array(std_filtering)
    mean_state_prediction = np.array(mean_state_prediction)
    std_state_prediction = np.array(std_state_prediction)
    mean_observation = np.array(mean_observation)
    std_observation = np.array(std_observation)
    loglikelihood = np.array(loglikelihood)
    ancestor_indices = np.array(ancestor_indices)

    output = SimpleNamespace(
        nu_weights=nu_weights, particles=particles, mean_filtering=mean_filtering,
        std_filtering=std_filtering, mean_state_prediction=mean_state_prediction,
        std_state_prediction=std_state_prediction, mean_observation=mean_observation,
        std_observation=std_observation, loglikelihood=loglikelihood, ancestor_indices=ancestor_indices,
    )
    return output

In [ ]:
N = 5000
initial_particles = np.random.normal(0, 1, N)
output = fully_adapted_pf(initial_particles, step_x, C, Q, R, seed=0, theta=1)

In [ ]:
np.mean(np.abs(x_data - output.mean_filtering)), np.mean(x_data - output.mean_filtering), np.var(x_data - output.mean_filtering)

In [ ]:
plt.plot(x_data - output.mean_filtering)

In [ ]:
plt.plot(x_data)
plt.plot(output.mean_filtering);

In [ ]:
plt.plot(output.std_filtering)

In [ ]:
# Estimate parameters (infer posterior p(theta|y_data)) using Particle Metropolis Hastings
theta_prior = scipy.stats.norm(0, 1)  # p(theta) = N(0, 1)
theta_rw_proposal = scipy.stats.norm(0, 0.1)  # q(theta'|theta) = N(0, 0.1)
M = 100  # Number of PMH runs

In [ ]:
def particle_metropolis_hastings(initial_theta, random_walk_proposal, step_x, C, Q, R, seed=seed):
    np.random.seed(seed)
    theta = initial_theta
    for m in range(M):
        theta = theta + random_walk_proposal.rvs()
        output = fully_adapted_pf(initial_particles, step_x, C, Q, R, seed=None, theta=theta)
        acceptance_ratio = 

In [ ]:
initial_theta = 0.5  # theta_prior.rvs()
particle_metropolis_hastings(initial_theta, theta_rw_proposal, step_x, C, Q, R, seed=0)

# Problem 2

# Problem 3

# Problem 4

# Problem 5

# Problem 6